In [5]:
import pandas as pd
import os

In [316]:
# Load the Excel file
file_path = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales\invoice_Annexure_408_09042025_1744207092917.xlsx"

# Load the Summary sheet (or check actual sheet name)
df = pd.read_excel(file_path, sheet_name="Summary")

# print("Summary Tab:")
# print(df.head())


<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\hp\AppData\Local\Temp\ipykernel_1760\4214857672.py:2: SyntaxWarning: invalid escape sequence '\D'
  file_path = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales\invoice_Annexure_408_09042025_1744207092917.xlsx"


In [277]:
# folder_name = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales"
# file_names = ["invoice_Annexure_408_09042025_1744207092917","invoice_Annexure_4780_09042025_1744206715940", "invoice_Annexure_180796_09042025_1744207214805",
#             "invoice_Annexure_180801_09042025_1744199507239","invoice_Annexure_231392_09042025_1744199484679","invoice_Annexure_313834_09042025_1744207163742",
#             "invoice_Annexure_330114_09042025_1744207176505","invoice_Annexure_444423_09042025_1744195095444","invoice_Annexure_445384_09042025_1744199421232",
#             "invoice_Annexure_570268_09042025_1744199329588","invoice_Annexure_593472_09042025_1744195195702","invoice_Annexure_600278_09042025_1744206940598"]
# for file_name in file_names:
#     All_DF=pd.read_excel(f"{folder_name}\{file_name}.xlsx")
#     print(All_DF)

In [528]:
def extract_summary_info(df):
    # Clean up
    df = df.dropna(how='all')  # Remove fully empty rows
    df = df.reset_index(drop=True)

    # Extract fields by searching
    info = {
        "Restaurant_ID": df.iloc[3, 1].replace("Rest. ID - ", ""),# if len(df) > 6 and isinstance(df.iloc[6,1], str) else None,
        "Brand": df.iloc[0, 1],
        "Location": df.iloc[1, 1],
        "City": df.iloc[2, 1],
        "GSTIN": df.iloc[4, 1].replace("GSTIN  - ", ""),
        "Payout_Period": df.iloc[6, 2],
        "Settlement_Date": df.iloc[7, 2], 
        "Total_Payout": df.iloc[8, 2].replace("₹", "").replace(",", "").strip(),
        "Total_Orders": df.iloc[9, 2] ,
        "Bank_UTR": df.iloc[10, 2],
    }

    return pd.DataFrame([info])
extract_summary_info(df)



,Restaurant_ID,Brand,Location,City,GSTIN,Payout_Period,Settlement_Date,Total_Payout,Total_Orders,Bank_UTR
0,408,Alpha Biryani,Whitefield,Bangalore,29ABNFM9601R1Z9,01 April - 05 April,08 April,6818.6,26,AXISCN0957979233


In [532]:
summary_data = []
folder_path = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales"

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        full_path = os.path.join(folder_path, file)
        try:
            df = pd.read_excel(full_path, sheet_name="Summary", engine="openpyxl")
            summary_df = extract_summary_info(df)
            summary_df["Source_File"] = file 
            summary_data.append(summary_df)
            # print(summary_data)
        except Exception as e:
            print(f"Error processing {file}: {e}")

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\hp\AppData\Local\Temp\ipykernel_1760\3362008785.py:2: SyntaxWarning: invalid escape sequence '\D'
  folder_path = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales"


In [534]:
# Combine all extracted data
final_summary_df = pd.concat(summary_data, ignore_index=True)
# print(final_summary_df)
#Fetching required columns
selected_columns = final_summary_df.loc[:,["Restaurant_ID","Brand","Location","City","GSTIN","Payout_Period","Settlement_Date","Total_Payout","Total_Orders","Bank_UTR"]]
selected_columns.head()

,Restaurant_ID,Brand,Location,City,GSTIN,Payout_Period,Settlement_Date,Total_Payout,Total_Orders,Bank_UTR
0,180796,Biryani Box,Whitefield,Bangalore,29ABNFM9601R1Z9,01 April - 05 April,08 April,1709.95,8,AXISCN0957982367
1,180801,Roj Ka Khana(Daily Meals),Whitefield,Bangalore,29ABNFM9601R1Z9,01 April - 05 April,08 April,26087.91,124,AXISCN0957908193
2,231392,Chinese Chaska,Whitefield,Bangalore,29ABNFM9601R1Z9,01 April - 05 April,08 April,11162.8,50,AXISCN0957908771
3,313834,Biryani Hub,Whitefield,Bangalore,29ABNFM9601R1Z9,01 April - 05 April,08 April,5566.3,40,AXISCN0957980584
4,330114,Ramji Cafe,Whitefield,Bangalore,29ABNFM9601R1Z9,01 April - 05 April,08 April,7287.09,39,AXISCN0957981078


In [536]:
selected_columns.count()

Restaurant_ID      12
Brand              12
Location           12
City               12
GSTIN              12
Payout_Period      12
Settlement_Date    12
Total_Payout       12
Total_Orders       12
Bank_UTR           12
dtype: int64

In [538]:
# Extract the Brand details and payout details  from summary Tab of Annexure

Grouped_data=selected_columns.groupby("Brand").agg(Total_Payout=('Total_Payout','sum'),Total_Orders=('Total_Orders','sum'))
Sorted_data=Grouped_data.sort_values(by='Total_Payout', ascending=True).reset_index()

Sorted_data

,Brand,Total_Payout,Total_Orders
0,Koolerz,102.98,1
1,Chinese Chaska,11162.8,50
2,Dilli Chaat,14011.42,74
3,Biryani Box,1709.95,8
4,Mann Food Court,26070.88,120
5,Roj Ka Khana(Daily Meals),26087.91,124
6,Uruvalu Biryani,2807.7,16
7,Grill Factory,36860.43,158
8,Biryani Hub,5566.3,40
9,Dilli Darbar,61563.0,227


In [542]:
def extract_brand_name(df):
    df = df.dropna(how='all').reset_index(drop=True)
    if len(df) > 3 and pd.notna(df.iloc[3, 1]):
        return df.iloc[0, 1]  # Brand name
    return None
# extract_brand_name(df)

'Dilli Chaat'

In [544]:
import pandas as pd
import os

folder_path = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales"
payout_data = []
order_level_data = []

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        try:
            # Extract brand name from Summary tab
            summary_df = pd.read_excel(file_path, sheet_name="Summary", engine="openpyxl")
            brand = extract_brand_name(summary_df)
            # print(brand)

            # Extract Payout BreakUp tab
            payout_df = pd.read_excel(file_path, sheet_name="Payout Breakup", engine="openpyxl")
            payout_df["Brand"] = brand
            # payout_df["Source_File"] = file
            payout_df["Total Customer Paid"]=payout_df.iloc[3:4,5:6]
            payout_df['Swiggy Fee']=payout_df.iloc[9:10,5:6]
            payout_df['Customer Complaints & Cancellation Charges']=payout_df.iloc[21:22,5:6]
            payout_df['Other Charges and Refunds']=payout_df.iloc[25:26,5:6]
            payout_df['Total Taxes']=payout_df.iloc[27:28,5:6]
            payout_df['Net Payout']=payout_df.iloc[32:33,5:6]
            
            payout_data.append(payout_df)

            # Extract Order Level tab
            # order_df1=pd.DataFrame()
            order_df = pd.read_excel(file_path, sheet_name="Order Level", engine="openpyxl")
            order_df["Brand"] = brand
            order_df["Order ID"]=order_df.iloc[2:,0]
            order_df["Order Date"]=order_df.iloc[2:,2]
            order_df["Customer Paid"]=order_df.iloc[2:,15]
            order_df["Swiggy Commission / Platform Fee"]=order_df.iloc[2:,29]
            order_df["Restaurant Earnings"]=order_df.iloc[2:,37]
            order_df["Discounts (Restaurant/Platform)"]=order_df.iloc[2:,12]
            order_df["Taxes"]=order_df.iloc[2:,36]
            order_df["Cancellation Charges / Refunds"]=order_df.iloc[2:,24]
            order_df["Payment Mode"]=order_df.iloc[2:,5]
            
            order_level_data.append(order_df)
            # print(order_level_data)
            # print(order_df.columns)
            # print(order_df1.head())
            # print(order_df)
            
        except Exception as e:
            print(f"Error in file {file}: {e}")

<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\hp\AppData\Local\Temp\ipykernel_1760\4195077166.py:4: SyntaxWarning: invalid escape sequence '\D'
  folder_path = "E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Payout Summary & Order Level Sales"


In [546]:
# Combine all into one DataFrame
all_payouts_df = pd.concat(payout_data, ignore_index=True)
# print(all_payouts_df)
# remove empty rows if any
all_payouts_df.dropna(how='all', inplace=True)

In [548]:
# Extract Payout Break Up for all files and add brand summary
all_payouts_df.groupby('Brand').agg(
    Total_Customer_Paid=('Total Customer Paid', 'sum'),
    Swiggy_Fee=('Swiggy Fee', 'sum'),
    Customer_Complaints_Cancellation_Charges=('Customer Complaints & Cancellation Charges', 'sum'),
    Other_Charges_Refunds=('Other Charges and Refunds', 'sum'),
    Total_Taxes=('Total Taxes', 'sum'),
    Net_Payout=('Net Payout', 'sum')
).reset_index()


,Brand,Total_Customer_Paid,Swiggy_Fee,Customer_Complaints_Cancellation_Charges,Other_Charges_Refunds,Total_Taxes,Net_Payout
0,Alpha Biryani,10468.97,-3141.85,0.0,0.0,-508.51,6818.6
1,Biryani Box,2722.81,-880.59,0.0,0.0,-132.26,1709.95
2,Biryani Hub,8912.91,-2913.66,0.0,0.0,-432.9,5566.3
3,Chinese Chaska,17376.96,-5111.42,-271.78,0.0,-830.6,11162.8
4,Dilli Chaat,21524.39,-6467.35,0.0,0.0,-1045.48,14011.42
5,Dilli Darbar,96690.98,-30192.69,-250.54,0.0,-4684.16,61563.0
6,Grill Factory,56583.11,-16974.5,0.0,0.0,-2748.22,36860.43
7,Koolerz,154.11,-43.65,0.0,0.0,-7.49,102.98
8,Mann Food Court,39378.15,-11241.32,-160.66,0.0,-1904.74,26070.88
9,Ramji Cafe,10959.11,-3139.51,0.0,0.0,-532.3,7287.09


In [550]:
# Extract Order Level Details for all files and add brand summary 

order_df_all = pd.concat(order_level_data, ignore_index=True)
order_df_all = order_df_all.dropna(how='all').reset_index(drop=True)
# print(order_df_all)
grouped_df = order_df_all.groupby(['Brand', 'Order ID']).agg({
    'Customer Paid': 'sum',
    'Swiggy Commission / Platform Fee': 'sum',
    'Restaurant Earnings': 'sum',
    'Discounts (Restaurant/Platform)': 'sum',
    'Taxes': 'sum',
    'Cancellation Charges / Refunds': 'sum'
}).reset_index()
grouped_df

,Brand,Order ID,Customer Paid,Swiggy Commission / Platform Fee,Restaurant Earnings,Discounts (Restaurant/Platform),Taxes,Cancellation Charges / Refunds
0,Alpha Biryani,202498434156623,165.9,46.98,110.84,80,8.06,0
1,Alpha Biryani,202547157961802,263.56,74.63,176.1,125,12.8,0
2,Alpha Biryani,202560186008853,186.9,52.94,124.9,80,9.08,0
3,Alpha Biryani,202572805829234,383.16,108.52,256.05,163.08,18.61,0
4,Alpha Biryani,202572847613980,276.17,78.22,184.55,79.99,13.41,0
...,...,...,...,...,...,...,...,...
878,Uruvalu Biryani,202663725914625,291.9,82.68,195.06,150,14.18,0
879,Uruvalu Biryani,202726325174878,184.8,52.33,123.49,100,8.98,0
880,Uruvalu Biryani,202726516346620,305.56,86.53,204.18,125,14.84,0
881,Uruvalu Biryani,202745605872120,363.3,102.89,242.78,150,17.65,0


In [552]:
# # Extract Details from Commission Invoice pdf and all details in excel files
# import os
# # !pip install pdfplumber
# # import pdfplumber
# !pip install PyPDF2
# import PyPDF2
# # from tabula import read_pdf
# import pandas as pd

# folder_path = r"E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Commission Invoices" 
# data = []

# for file in os.listdir(folder_path):
#     if file.endswith(".pdf"):
#         file_path = os.path.join(folder_path, file)
#         df_list = read_pdf(file_path, stream=True)
#         print(df_list[0])
# #         with pdfplumber.open(file_path) as pdf:
# #             text = ""
# #             for page in pdf.pages:
# #                 text += page.extract_text() + "\n"  # Extract text from each page

# #         # Example: extract specific values from text using string methods or regex
# #         brand = "Unknown"
# #         gstin = "Not Found"
# #         invoice_no = "Not Found"
# #         invoice_date = "Not Found"

# #         if "Restaurant Name:" in text:
# #             lines = text.splitlines()
# #             for line in lines:
# #                 if "Restaurant Name:" in line:
# #                     brand = line.split(":")[-1].strip()
# #                 elif "GSTIN:" in line:
# #                     gstin = line.split(":")[-1].strip()
# #                 elif "Invoice Number:" in line:
# #                     invoice_no = line.split(":")[-1].strip()
# #                 elif "Invoice Date:" in line:
# #                     invoice_date = line.split(":")[-1].strip()

# #         data.append({
# #             "File Name": file,
# #             "Brand": brand,
# #             "GSTIN": gstin,
# #             "Invoice Number": invoice_no,
# #             "Invoice Date": invoice_date
# #         })

# # # Convert to DataFrame
# # pdf_df = pd.DataFrame(data)
# # print(pdf_df)

In [563]:
# !pip install pypdf
from pypdf import PdfReader

folder_path = r"E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Commission Invoices" 
data = []

for file in os.listdir(folder_path):
    if file.endswith(".pdf"):
        file_path = os.path.join(folder_path, file)
        # Load the PDF file
        reader = PdfReader(file_path)  # Replace with your PDF file path
        
        # Loop through all pages and extract text
        for i, page in enumerate(reader.pages):
            text = page.extract_text()
            print(f"--- Page {i + 1} ---")
            print(text)
            print("\n")

--- Page 1 ---
TAX INVOICE
Invoice From : Swiggy Limited (formerly known 
as Swiggy Private Limited and 
Bundl Technologies Private 
Limited)
IRN : 67b9bd5fe06a5ff1e282d7fbc5c36829cbbde240761468b6aeac4a1f3aa7d120
PAN : AAFCB7707D Invoice Number : 250409FS29011371
Email ID : invoice@swiggy.in Invoice Date : 2025-04-09 19:30:13
GSTIN : 29AAFCB7707D1ZQ Service Period : 01/04/2025 to 05/04/2025
Address : No 55, Sy No 8 to 14, I & J block, 
Ground Floor, Embassy Tech 
village, Outer Ring Road, 
Devarbisanahalli, Varthur, 
Bengaluru - 560103.
Invoice Type : INV
Original Invoice 
No:
-
Original Invoice 
Date:
-
Pincode : 560103 PO Number : -
State Code : 29 QR CODE :
Invoice To :
Legal Name : MANN HOSPITALITY LLP
Restaurant / Store Name :Biryani Box
Restaurant / Store ID : 180796
GSTIN : 29ABNFM9601R1Z9
Address : SY No 166, Mann Food Court, Ak 
Gopalan Colony Road, Near Hope 
Farm, Whitefield Bangalore - 
560066
City : Bangalore
Pincode : 560066
State Code : 29
Sr. 
no.
Description HSN
Unit o

In [555]:
import os
import pdfplumber
import pandas as pd

# Path to your folder with PDFs
pdf_folder = r"E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Commission Invoices"
output_excel = r"E:\Downloads\Swiggy_Tax_Invoice_Summary.xlsx"

all_data = []

for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file)
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # Skip header row
                    df['Source File'] = file
                    all_data.append(df)

# Combine all tables
final_df = pd.concat(all_data, ignore_index=True)

# Clean column names if needed
final_df.columns = [col.strip() for col in final_df.columns]

# Save to Excel
final_df.to_excel(output_excel, index=False)
print(f"Saved extracted data to: {output_excel}")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [569]:
import os
import re
import pdfplumber
import pandas as pd

folder_path = r"E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Commission Invoices"  # Replace this
extracted_data = []

# Function to safely extract a value using regex
def extract_with_regex(pattern, text, default=''):
    match = re.search(pattern, text, re.IGNORECASE)
    return match.group(1).strip() if match else default

for file in os.listdir(folder_path):
    if file.endswith(".pdf"):
        file_path = os.path.join(folder_path, file)
        with pdfplumber.open(file_path) as pdf:
            full_text = ""
            for page in pdf.pages:
                full_text += page.extract_text() + "\n"

            # Extract fields
            invoice_date = extract_with_regex(r"(?:Invoice Date|Date of Invoice|Dated|Invoice Dt\.?)?\s*[:\-]?\s*(\d{4}-\d{2}-\d{2})",
            full_text)
            invoice_number = extract_with_regex(r"Invoice Number\s*[:\-]?\s*(\S+)", full_text)
            hsn = extract_with_regex(r"HSN\s*[:\-]?\s*(\d+)", full_text)
            cgst_rate = extract_with_regex(r"CGST\s+Rate\s*[:\-]?\s*([\d.]+)%?", full_text)
            cgst_amount = extract_with_regex(r"CGST\s+Amount\s*[:\-]?\s*₹?([\d,.]+)", full_text)
            sgst_rate = extract_with_regex(r"SGST\s+Rate\s*[:\-]?\s*([\d.]+)%?", full_text)
            sgst_amount = extract_with_regex(r"SGST\s+Amount\s*[:\-]?\s*₹?([\d,.]+)", full_text)
            igst_rate = extract_with_regex(r"IGST\s+Rate\s*[:\-]?\s*([\d.]+)%?", full_text)
            igst_amount = extract_with_regex(r"IGST\s+Amount\s*[:\-]?\s*₹?([\d,.]+)", full_text)
            total_amount = extract_with_regex(r"Total Amount\s*[:\-]?\s*₹?([\d,.]+)", full_text)
            pan = extract_with_regex(r"PAN\s*[:\-]?\s*([A-Z]{5}\d{4}[A-Z])", full_text)
            swiggy_gstin = extract_with_regex(r"Swiggy\s+GSTIN\s*[:\-]?\s*([0-9A-Z]{15})", full_text)
            mann_gstin = extract_with_regex(r"Restaurant\s+GSTIN\s*[:\-]?\s*([0-9A-Z]{15})", full_text)

            extracted_data.append({
                "file_name": file,
                "invoice_date": invoice_date,
                "invoice_number": invoice_number,
                "hsn": hsn,
                "cgst_rate": cgst_rate,
                "cgst_amount": cgst_amount,
                "sgst_rate": sgst_rate,
                "sgst_amount": sgst_amount,
                "igst_rate": igst_rate,
                "igst_amount": igst_amount,
                "total_amount": total_amount,
                "pan": pan,
                "swiggy_gstin": swiggy_gstin,
                "mann_gstin": mann_gstin,
            })

# Convert to DataFrame
df = pd.DataFrame(extracted_data)
df.to_excel("extracted_invoice_summary.xlsx", index=False)
print("Extracted data saved to 'extracted_invoice_summary.xlsx'")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

✅ Extracted data saved to 'extracted_invoice_summary.xlsx'


In [571]:
pd.read_excel('extracted_invoice_summary.xlsx')



,file_name,invoice_date,invoice_number,hsn,cgst_rate,cgst_amount,sgst_rate,sgst_amount,igst_rate,igst_amount,total_amount,pan,swiggy_gstin,mann_gstin
0,taco_Tax_Invoice_180796_09042025_250409FS29011...,2025-04-09,250409FS29011371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
1,taco_Tax_Invoice_180801_09042025_250409FS29007...,2025-04-09,250409FS29007230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
2,taco_Tax_Invoice_231392_09042025_250409FS29007...,2025-04-09,250409FS29007041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
3,taco_Tax_Invoice_313834_09042025_250409FS29011...,2025-04-09,250409FS29011069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
4,taco_Tax_Invoice_330114_09042025_250409FS29011...,2025-04-09,250409FS29011158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
5,taco_Tax_Invoice_408_09042025_250409FS29010820...,2025-04-09,250409FS29010820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
6,taco_Tax_Invoice_444423_09042025_250409FS29002...,2025-04-09,250409FS29002733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
7,taco_Tax_Invoice_445384_09042025_250409FS29006...,2025-04-09,250409FS29006872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
8,taco_Tax_Invoice_4780_09042025_250409FS2900915...,2025-04-09,250409FS29009158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN
9,taco_Tax_Invoice_570268_09042025_250409FS29006...,2025-04-09,250409FS29006449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFCB7707D,NaN,NaN


In [593]:
import os
import re
import pandas as pd
from pypdf import PdfReader

def extract_with_regex(pattern, text, group=1):
    match = re.search(pattern, text, re.IGNORECASE)
    return match.group(group).strip() if match else None

def parse_invoice_pdf(file_path):
    reader = PdfReader(file_path)
    full_text = "\n".join([page.extract_text() for page in reader.pages])

    data = {
        "file_name": os.path.basename(file_path),
        "invoice_date": extract_with_regex(r"(\d{4}-\d{2}-\d{2})", full_text),
        "invoice_number": extract_with_regex(r"Invoice Number\s*[:\-]?\s*(\S+)", full_text),
        "irn": extract_with_regex(r"IRN\s*:?\s*([A-Z0-9]+)", full_text),
        "mann_gstin" :next((gstin for gstin in re.findall(r"GSTIN\s*:?\s*([0-9A-Z]{15})", full_text)), None)
,
        "swiggy_gstin": extract_with_regex(r"GSTIN of Recipient\s*:?\s*([0-9A-Z]+)", full_text),
        "pan": extract_with_regex(r"PAN\s*:?\s*([A-Z]{5}[0-9]{4}[A-Z])", full_text),
        "fy_year": extract_with_regex(r"FY\s*:?\s*(\d{4}-\d{2})", full_text),
        "total_amount": extract_with_regex(r" Grand Total\s*Rs\.?\s*([\d,]+\.\d+)", full_text),
        "grand_total": extract_with_regex(r"Grand Total\s*Rs\.?\s*([\d,]+\.\d+)", full_text),
        "cgst_amount": extract_with_regex(r"CGST Rate\s*Rs\.?\s*([\d,]+\.\d+)", full_text),
        "sgst_amount": extract_with_regex(r"SGST\s*Rs\.?\s*([\d,]+\.\d+)", full_text),
        "igst_amount": extract_with_regex(r"IGST\s*Rs\.?\s*([\d,]+\.\d+)", full_text),
        "description": extract_with_regex(r"Description\s*:?\s*(.*?)\s*HSN", full_text),
        "hsn": extract_with_regex(r"HSN\s*:?\s*(\d+)", full_text),
        "unit_price": extract_with_regex(r"Unit Price\s*:?\s*Rs\.?\s*([\d,]+\.\d+)", full_text),
        "quantity": extract_with_regex(r"Quantity\s*:?\s*([\d.]+)", full_text),
        "unit_of_measure": extract_with_regex(r"Unit of Measure\s*:?\s*(\w+)", full_text),
        # Add more fields as needed...
    }

    return data

# Directory containing PDF files
folder_path = r"E:\Downloads\Cleaning of Data & Merging into single excel\Cleaning of Data & Merging into single excel\Commission Invoices"

records = []
for file in os.listdir(folder_path):
    if file.endswith(".pdf"):
        file_path = os.path.join(folder_path, file)
        record = parse_invoice_pdf(file_path)
        records.append(record)

df = pd.DataFrame(records)
df.to_excel("commission_invoice_summary.xlsx", index=False)


In [585]:
pd.read_excel('commission_invoice_summary.xlsx')

,file_name,invoice_date,invoice_number,irn,mann_gstin,swiggy_gstin,pan,fy_year,total_amount,grand_total,cgst_amount,sgst_amount,igst_amount,description,hsn,unit_price,quantity,unit_of_measure
0,taco_Tax_Invoice_180796_09042025_250409FS29011...,2025-04-09,250409FS29011371,67b9bd5fe06a5ff1e282d7fbc5c36829cbbde240761468...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,taco_Tax_Invoice_180801_09042025_250409FS29007...,2025-04-09,250409FS29007230,9b435519a6a657136ee8760e596dbdb19b5ca8a519c3a2...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,taco_Tax_Invoice_231392_09042025_250409FS29007...,2025-04-09,250409FS29007041,9ab1e0b95855cfbdae1e78ac0e8c2e1bd17d94a04c47fe...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,taco_Tax_Invoice_313834_09042025_250409FS29011...,2025-04-09,250409FS29011069,45cbbf52c36731bc44692a6fbc18fc02dfab0636f04b5e...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,taco_Tax_Invoice_330114_09042025_250409FS29011...,2025-04-09,250409FS29011158,423722ba0c134895f913eb5abc69ac6d2111509090c2be...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,taco_Tax_Invoice_408_09042025_250409FS29010820...,2025-04-09,250409FS29010820,766f2de785453717bd7194b0335b80c7cd39b341518a9e...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,taco_Tax_Invoice_444423_09042025_250409FS29002...,2025-04-09,250409FS29002733,5eb5f6814b2f6a1104b7ac87a48a39da41cc4d7fc03bde...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,taco_Tax_Invoice_445384_09042025_250409FS29006...,2025-04-09,250409FS29006872,952606c05d4180442f294916e09ca9ba6daff9de3146ff...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,taco_Tax_Invoice_4780_09042025_250409FS2900915...,2025-04-09,250409FS29009158,c07bcd8df4a06a6a2d1e1cd74cd1315c5bc65a0d4b5458...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,taco_Tax_Invoice_570268_09042025_250409FS29006...,2025-04-09,250409FS29006449,dfcf6bfb99453ec1dedfd49aa20819e06a67c13a36d618...,NaN,NaN,AAFCB7707D,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
